# Lounge Eligibility Lookup — Data Science Project
**Author:** Hlangalezwe Zulu  
**Purpose:** Build a reusable lookup table and demonstration notebook to estimate lounge eligibility (Tier 1: Concorde Room, Tier 2: First Lounge, Tier 3: Club Lounge) for British Airways flight schedules.  

This Colab notebook is structured to be runnable in Google Colab. Upload the provided Excel schedule (`British Airways Summer Schedule Dataset - Forage Data Science Task 1.xlsx`) when prompted.

---
## Instructions to run in Google Colab
1. Open this notebook in Google Colab (File > Upload notebook).  
2. Upload the BA schedule Excel file when prompted by the file upload cell, or mount Google Drive and place the file there.  
3. Run cells sequentially. The notebook will produce lookup tables, sample applications, and export results to Excel/CSV.
---

In [ ]:

# Upload the dataset (use this cell in Google Colab)
from google.colab import files
uploaded = files.upload()  # Use this to upload the Excel file directly
for fn in uploaded.keys():
    print('Uploaded file:', fn)
# After upload, set the filename here (or it will try to detect the first .xlsx)
import glob, os
xlsx_files = [f for f in os.listdir('.') if f.endswith('.xlsx')]
xlsx_files


In [ ]:

# Imports and helper functions
import pandas as pd
import numpy as np
from datetime import datetime, time

# Helper: classify time of day
def classify_time_of_day(dep_time_str):
    # Accept strings like '05:30' or integers like 530 or Excel time floats.
    try:
        if isinstance(dep_time_str, (int, float)):
            # assume HHMM e.g. 530 or 1530
            s = str(int(dep_time_str)).zfill(4)
            hh = int(s[:-2]); mm = int(s[-2:])
            t = time(hh, mm)
        else:
            # try parse
            t = datetime.strptime(str(dep_time_str), '%H:%M').time()
    except Exception:
        # fall back to noon
        t = time(12,0)
    if time(5,0) <= t <= time(9,59):
        return 'Morning'
    if time(10,0) <= t <= time(15,59):
        return 'Afternoon'
    return 'Evening'

# Function to apply lookup table and estimate lounge pax
def estimate_lounge_pax(df, lookup):
    # df must have columns: 'TotalSeats' or calculate from seat counts; 'Haul', 'DepTimeClass'
    results = []
    for _, row in df.iterrows():
        haul = row.get('Haul','Short-haul')
        tod = row.get('DepTimeClass','Afternoon')
        total = row.get('TotalPax', np.nan)
        if pd.isna(total):
            # attempt to sum available seat columns
            seat_cols = [c for c in df.columns if 'seats' in c.lower() or 'seat' in c.lower() or c.lower() in ['totalpax','total_pax','total']]
            found = False
            for sc in seat_cols:
                if not pd.isna(row.get(sc)):
                    total = row.get(sc)
                    found = True
                    break
            if not found:
                total = 100  # default
        key = (haul, tod)
        if key not in lookup:
            # fallback to haul-only or default
            if (haul, 'Afternoon') in lookup:
                pct = lookup[(haul, 'Afternoon')]
            else:
                pct = lookup.get(('Short-haul','Afternoon'), {'Tier1':0.002,'Tier2':0.03,'Tier3':0.10})
        else:
            pct = lookup[key]
        est = { 'Flight': row.get('Flight',''), 'Haul': haul, 'DepTimeClass': tod, 'TotalPax': total }
        for tier, p in pct.items():
            est[tier] = round(total * p)
        results.append(est)
    return pd.DataFrame(results)


In [ ]:

# Define the lookup table (percentages as decimals)
lookup = {
    ('Long-haul','Morning'):  {'Tier1':0.011, 'Tier2':0.088,  'Tier3':0.275},
    ('Long-haul','Afternoon'):{'Tier1':0.010, 'Tier2':0.080,  'Tier3':0.250},
    ('Long-haul','Evening'):  {'Tier1':0.0105,'Tier2':0.084,  'Tier3':0.2625},
    ('Short-haul','Morning'):{'Tier1':0.0022,'Tier2':0.033,  'Tier3':0.110},
    ('Short-haul','Afternoon'):{'Tier1':0.0020,'Tier2':0.030,'Tier3':0.100},
    ('Short-haul','Evening'):{'Tier1':0.0021,'Tier2':0.0315,'Tier3':0.105},
}
print('Lookup table defined.')


In [ ]:

# Load the schedule file (adjust filename if different)
import os, glob
xlsx_candidates = [f for f in os.listdir('.') if f.lower().endswith('.xlsx')]
if len(xlsx_candidates)==0:
    print('No .xlsx file found - please upload the schedule using the upload cell above.')
else:
    fn = xlsx_candidates[0]
    print('Loading', fn)
    df_raw = pd.read_excel(fn)
    display(df_raw.head())
    df = df_raw.copy()
    df.columns = [c.strip() for c in df.columns]
    possible = [c for c in df.columns if 'total' in c.lower() or 'pax' in c.lower() or 'pass' in c.lower()]
    if possible:
        df['TotalPax'] = df[possible[0]]
    else:
        seat_cols = [c for c in df.columns if any(k in c.lower() for k in ['seat','economy','business','first'])]
        if seat_cols:
            df['TotalPax'] = df[seat_cols].sum(axis=1)
        else:
            df['TotalPax'] = 150
    if 'Haul' not in df.columns:
        if 'Region' in df.columns:
            df['Haul'] = df['Region'].apply(lambda x: 'Long-haul' if str(x).strip().lower() not in ['europe'] else 'Short-haul')
        elif 'Destination' in df.columns:
            df['Haul'] = df['Destination'].apply(lambda x: 'Short-haul' if str(x).strip().lower() in ['london','paris','berlin','amsterdam'] else 'Long-haul')
        else:
            df['Haul'] = df['Flight'].apply(lambda x: 'Long-haul' if str(x).strip().lower().startswith('ba2') else 'Short-haul')
    if 'DepTime' in df.columns:
        df['DepTimeClass'] = df['DepTime'].apply(classify_time_of_day)
    elif 'SchedDep' in df.columns:
        df['DepTimeClass'] = df['SchedDep'].apply(classify_time_of_day)
    else:
        df['DepTimeClass'] = 'Afternoon'
    display(df[['Flight','Haul','DepTimeClass','TotalPax']].head())


In [ ]:

# Apply estimation to the preprocessed df and save results
est_df = estimate_lounge_pax(df, lookup)
display(est_df.head(20))
est_df.to_csv('lounge_estimates.csv', index=False)
est_df.to_excel('lounge_estimates.xlsx', index=False)
print('Saved lounge_estimates.csv and lounge_estimates.xlsx in the working directory.')


In [ ]:

# Visualization (matplotlib)
import matplotlib.pyplot as plt
totals = est_df[['Tier1','Tier2','Tier3']].sum()
plt.figure(figsize=(6,4))
totals.plot(kind='bar')
plt.title('Estimated Total Eligible Passengers by Tier (Sample)')
plt.ylabel('Passengers')
plt.xlabel('Tier')
plt.tight_layout()
plt.show()


---
## Conclusions & Next Steps

- This notebook provides a reusable pipeline to apply the **Lounge Eligibility Lookup** to any future BA schedule.  
- **Personalized** by Hlangalezwe Zulu, it is ready to run in Colab and export results for reporting.  

**Next steps you may want to add:**  
1. Tune lookup percentages using historical lounge swipe or boarding data.  
2. Add capacity modelling (hourly/daily peaks) and overlap analysis.  
3. Connect to an API or data pipeline to run this regularly for updated schedules.
---
